# Chap07.4 - 텐서플로 추상화와 간소화, TF-Slim


## 7.5 TF-Slim

[**TF-Slim**](https://github.com/tensorflow/tensorflow/tree/r1.8/tensorflow/contrib/slim)은 텐서플로를 가볍게(?) 사용할 수 있는 텐서플로의 확장된 추상화 라이브러리이며, 복잡한 모델을 빠르고 직관적으로 정의하고 학습할 수 있다. TF-Slim은 텐서플로 내에 포함되어 있으므로 별도의 설치를 하지 않아도 된다. 

TF-Slim의 추상화는 모두 [**CNN**](http://excelsior-cjh.tistory.com/152?category=940399)(Convolutional Neural Network)에 관한 것이다. CNN의 경우 동일한 합성곱 계층(Convolution Layer)을 여러번 재사용하는 [보일러플레이트](https://en.wikipedia.org/wiki/Boilerplate_code)(boilerplate code)코드가 많다. 이러한 코드가 많을 경우 모델이 복잡해질 뿐만아니라 가독성 또한 떨어지게 된다. TF-Slim은 이러한 복잡한 CNN 모델을 High-Level, 추상화, arument scoping 등을 이용해 깔끔하게 작성할 수 있게 해준다. 

또한, TF-Slim은 자체적인 모델을 생성하고 학습할 수 있을 뿐만아니라, 사전에 학습된 모델인 [VGG](https://github.com/tensorflow/tensorflow/blob/r1.8/tensorflow/contrib/slim/python/slim/nets/vgg.py), [AlexNet](https://github.com/tensorflow/tensorflow/blob/r1.8/tensorflow/contrib/slim/python/slim/nets/alexnet.py), [Inception](https://github.com/tensorflow/tensorflow/blob/r1.8/tensorflow/contrib/slim/python/slim/nets/inception.py) 등을 제공한다.

### 7.5.1 TF-Slim 기능 및 사용방법

#### Usage

TF-Slim을 사용하기 위해서는 다음의 코드로 TF-Slim을 임포트한다.

```python
import tensorflow.contrib.slim as slim
```

#### Variables

TF-Slim은 초기화(`initializer`), 정규화(`regularizer`), 디바이스(`device`)를 하나의 래퍼(Wrapper)로 정의하여 변수를 쉽게 만들 수 있다. 아래의 예제 코드는 L2 정규화와 CPU를 사용하는 절단정규분포(`truncated_normal_initializer()`)로 초기화한 가중치(변수) `weights`를 정의하는 코드이다.

```python
import tensorflow as tf
import tensorflow.contrib.slim as slim

weights = slim.variable('weights', 
                        shape=[10, 10, 3, 3],
                        initializer=tf.truncated_normal_initializer(stddev=0.1),
                        regularizer=slim.l2_regularizer(0.05),
                        device='/CPU:0')
```

#### Layers

TF-Slim을 이용하면 순수 텐서플로를 이용해서 CNN 모델을 구현하는 것보다 훨씬 짧은 코드로 구현할 수 있다. 특히 보일러플레이트 코드와 불필요한 중복을 제거할 수 있다. 먼저, 합성곱 계층을 TF-Slim을 이용해서 정의 해보자.

```python
# 샘플 코드
net = slim.conv2d(inputs, 64, [5, 5], padding='SAME',
                  weights_initalizer=tf.truncated_normal_initializer(stddev=0.01),
                  weights_regularizer=slim.l2_regularizer(0.0005), scop='conv1')
```

위의 샘플 코드에서 처럼 합성곱 연산, 가중치 초기화, 정규화, 활성화 함수 등을 한번에 설정해줄 수 있다.

그리고, TF-slim은 `repeat`을 이용해 동일한 합성곱 계층들을 한줄로 표현할 수 있다. 아래의 코드는 5개의 동일한 합성곱 계층을 
쌓은 것을 나타낸 코드이다.
```python
# 샘플코드 - 같은 합성곱 계층을 5번 쌓기
net = slim.conv2d(net, 128, [3, 3], scope='con1_1')
net = slim.conv2d(net, 128, [3, 3], scope='con1_2')
net = slim.conv2d(net, 128, [3, 3], scope='con1_3')
net = slim.conv2d(net, 128, [3, 3], scope='con1_4')
net = slim.conv2d(net, 128, [3, 3], scope='con1_5')
```

위의 코드를 `repeat()`을 이용해 한줄로 나타낼 수 있다. 단, `repeat`은 계층의 크기가 동일한 경우에만 사용 가능하다.

```python
# 샘플코드 - slim.repeat()을 이용해 한줄로 나타내기
net = slim.repeat(net, 5, slim.conv2d, 128, [3, 3], scope='con1')
```

만약, 형태가 다른 경우에는 `stack`을 이용해 나타낼 수 있다.

```python
# 샘플코드 - 형태가 다른 합성곱 계층을 5개 쌓기
net = slim.conv2d(net, 64, [3, 3], scope='con1_1')
net = slim.conv2d(net, 64, [1, 1], scope='con1_2')
net = slim.conv2d(net, 128, [3, 3], scope='con1_3')
net = slim.conv2d(net, 128, [1, 1], scope='con1_4')
net = slim.conv2d(net, 256, [3, 3], scope='con1_5')

# 샘플코드 - slim.stack()을 이용해 한줄로 나타내기
net = slim.stack(net, slim.conv2d, [(64, [3, 3]), (64, [1, 1]),
                                    (128, [3, 3]), (128, [1, 1]),
                                    (256, [3, 3])], scope='con')
```

TF-Slim에는 `slim.conv2d` 계층 뿐만아니라 아래의 표와 같이 다양한 계층을 사용할 수 있다.

| Layer                    | TF-Slim                                                      |
| ------------------------ | ------------------------------------------------------------ |
| BiasAdd                  | [slim.bias_add](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| BatchNorm                | [slim.batch_norm](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| Conv2d                   | [slim.conv2d](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| Conv2dInPlane            | [slim.conv2d_in_plane](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| Conv2dTranspose (Deconv) | [slim.conv2d_transpose](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| FullyConnected           | [slim.fully_connected](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| AvgPool2D                | [slim.avg_pool2d](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| Dropout                  | [slim.dropout](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| Flatten                  | [slim.flatten](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| MaxPool2D                | [slim.max_pool2d](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| OneHotEncoding           | [slim.one_hot_encoding](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| SeparableConv2           | [slim.separable_conv2d](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |
| UnitNorm                 | [slim.unit_norm](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py) |

### arg_scope

TF-Slim은 `arg_scope`이라는 스코프를 가지고 있는데, 이것을 이용하면 같은 스코프에 정의되어 있는 여러 계층에 같은 인자들을 한번에 전달할 수 있다. 아래의 샘플 코드는 `slim.arg_scope`을 이용해 패딩, 활성화함수, 초기화, 정규화를 `slim.conv2d` 계층에 동일하게 설정하는 코드이다.

```python
# 샘플코드 - arg_scope을 이용해 slim.conv2d에 같은 인자 설정하기
with slim.arg_scope([slim.conv2d],
                    padding='SAME',
                    activation_fn=tf.nn.elu,
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01)):
    inputs = tf.reshape(x, [-1, 28, 28, 1])
    
    net = slim.conv2d(inputs=inputs, num_outputs=32, kernel_size=[5, 5], scope='conv1')
    net = slim.max_pool2d(inputs=net, kernel_size=[2, 2], scope='pool1')
    net = slim.conv2d(net, 64, [5, 5], scope='conv2')
    net = slim.max_pool2d(net, [2, 2], scope='pool2')
    net = slim.flatten(net, scope='flatten3')
```

### 7.5.2 TF-Slim으로 MNIST 분류 CNN 모델 구현하기

TF-Slim의 사용방법을 알아보았으니, 이번에는 [Chap04 - 합성곱 신경망 CNN](http://excelsior-cjh.tistory.com/152?category=940399)에서 구현한 CNN모델을 TF-Slim을 이용해 구현해 보도록하자. 7.5.1에서 TF-Slim에 대한 각 기능을 샘플코드로 살펴보았기 때문에 별도의 코드 설명은 생략한다. 

TF-Slim을 이용해 구현한 MNIST 분류 CNN 모델의 전체코드는 다음과 같다.

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim

#######################
# 0. mnist 불러오기
def mnist_load():
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

    # Train - Image
    train_x = train_x.astype('float32') / 255
    # Train - Label(OneHot)
    train_y = tf.keras.utils.to_categorical(train_y, num_classes=10)

    # Test - Image
    test_x = test_x.astype('float32') / 255
    # Test - Label(OneHot)
    test_y = tf.keras.utils.to_categorical(test_y, num_classes=10)
    
    return (train_x, train_y), (test_x, test_y)


(train_x, train_y), (test_x, test_y) = mnist_load()


#######################
# 1. placeholder 정의
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
is_training = tf.placeholder(tf.bool)

########################
# 2. TF-Slim을 이용한 CNN 모델 구현
with slim.arg_scope([slim.conv2d],
                    padding='SAME',
                    activation_fn=tf.nn.elu,
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01)):
    inputs = tf.reshape(x, [-1, 28, 28, 1])
    
    net = slim.conv2d(inputs=inputs, num_outputs=32, kernel_size=[5, 5], scope='conv1')
    net = slim.max_pool2d(inputs=net, kernel_size=[2, 2], scope='pool1')
    net = slim.conv2d(net, 64, [5, 5], scope='conv2')
    net = slim.max_pool2d(net, [2, 2], scope='pool2')
    net = slim.flatten(net, scope='flatten3')
    
with slim.arg_scope([slim.fully_connected],
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01)):
    net = slim.fully_connected(net, 1024, activation_fn=tf.nn.relu, scope='fc3')
    net = slim.dropout(net, is_training=is_training, scope='dropout3')
    outputs = slim.fully_connected(net, 10, activation_fn=None)
    
########################
# 3. loss, optimizer, accuracy
# loss
cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=y_))
# optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# accuracy
correct_prediction = tf.equal(tf.argmax(outputs, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

########################
# 4. Hyper-Paramter 설정 및 데이터 설정
# Hyper Parameters
STEPS = 5000
MINI_BATCH_SIZE = 50

# tf.data.Dataset을 이용한 배치 크기 만큼 데이터 불러오기
dataset = tf.data.Dataset.from_tensor_slices(({"image": train_x}, train_y))
dataset = dataset.shuffle(100000).repeat().batch(MINI_BATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

########################
# Training & Testing
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 학습
    for step in range(STEPS):
        batch_xs, batch_ys = sess.run(next_batch)
        _, cost_val = sess.run([train_step, cross_entropy], feed_dict={x: batch_xs['image'], 
                                                                       y_: batch_ys, 
                                                                       is_training: True})
        
        if (step+1) % 500 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch_xs['image'],
                                                           y_: batch_ys, 
                                                           is_training: False})
            print("Step : {}, cost : {:.5f}, training accuracy: {:.5f}".format(step+1, cost_val, 
                                                                               train_accuracy))
            
    X = test_x.reshape([10, 1000, 28, 28])
    Y = test_y.reshape([10, 1000, 10])

    test_accuracy = np.mean(
            [sess.run(accuracy, feed_dict={x: X[i], 
                                           y_: Y[i], 
                                           is_training: False}) for i in range(10)])

print("test accuracy: {:.5f}".format(test_accuracy))

Step : 500, cost : 0.18137, training accuracy: 0.98000
Step : 1000, cost : 0.16081, training accuracy: 0.96000
Step : 1500, cost : 0.25697, training accuracy: 0.92000
Step : 2000, cost : 0.11345, training accuracy: 0.96000
Step : 2500, cost : 0.03990, training accuracy: 1.00000
Step : 3000, cost : 0.07193, training accuracy: 1.00000
Step : 3500, cost : 0.08555, training accuracy: 0.94000
Step : 4000, cost : 0.06140, training accuracy: 1.00000
Step : 4500, cost : 0.09817, training accuracy: 0.98000
Step : 5000, cost : 0.00845, training accuracy: 1.00000
test accuracy: 0.98220


### 7.5.3 TF-Slim으로 VGG-16 모델 구현하기

TF-Slim을 이용해 CNN 모델 중 [VGG](https://arxiv.org/abs/1409.1556)모델을 구현해 보도록 하자. VGG모델은 2014년에 발표된 모델이며, 2014년 [ISLVRC](http://www.image-net.org/challenges/LSVRC/)에서 2위를 차지한 모델이다. VGG는 계층의 수가 16일때와 19사이일 때 결과가 가장 좋으며, 이번 예제에서는 TF-Slim을 이용해 13개의 합성곱 계층(Conv layer)와 3개의 완전연결 계층(FC, Fully-Connected layer), VGG-16 모델을 구현해 본다. 이번 예제 코드는 단순히 모델만 구현하는 것이므로 데이터 입력 및 학습에 대한 코드는 포함하지 않았다.

<img src="./images/vgg.png" height="90%" width="90%"/>

위의 그림에서 보듯이 VGG-16은 동일한 계층을 여러번 쓰는것을 확인할 수 있다. 따라서, TF-Slim의 `slim.arg_scope`와 `slim.repeat`을 이용해 VGG-16을 구현할 수 있다.

```python
# https://github.com/tensorflow/tensorflow/tree/r1.8/tensorflow/contrib/slim
def vgg16(inputs):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                          activation_fn=tf.nn.relu,
                          weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                          weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
        net = slim.max_pool2d(net, [2, 2], scope='pool1')
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
        net = slim.max_pool2d(net, [2, 2], scope='pool2')
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
        net = slim.max_pool2d(net, [2, 2], scope='pool3')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
        net = slim.max_pool2d(net, [2, 2], scope='pool4')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
        net = slim.max_pool2d(net, [2, 2], scope='pool5')
        net = slim.fully_connected(net, 4096, scope='fc6')
        net = slim.dropout(net, 0.5, scope='dropout6')
        net = slim.fully_connected(net, 4096, scope='fc7')
        net = slim.dropout(net, 0.5, scope='dropout7')
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc8')
    return net
```

### 7.5.4 사전 학습된 VGG-16모델 사용하기

이번에는 사전 학습된 VGG 모델을 다운로드 받아 사용해보자. 아래의 예제코드는 이미지 url 경로(http://54.68.5.226/car.jpg) 를 `urlib`모듈을 이용해 읽은 다음, 이 이미지를 사전 학습된 VGG-16 모델을 가지고 분류하는 작업을 나타낸 코드이다. 

<img src="http://54.68.5.226/car.jpg" width="50%" height="50%"/>

먼저, [TensorFlow](https://github.com/tensorflow/)가 제공하는 모델들이 있는 GitHub 저장소를 클론한다.

```bash
git clone https://github.com/tensorflow/models
```

그런 다음 클론한 경로를 파이썬의 `sys` 모듈을 이용해 설정해준다.

```python
import sys
sys.path.append('<클론한 경로>' + '/models/research/slim')
```

그리고, 사전 학습된 VGG-16 모델을 [download.tensorflow.org](http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz)에서 다운로드 받은 후 적당한 위치에 압축을 풀어주고 이것을 `target_dir`로 지정한다.

```python
target_dir = '<vgg 사전학습된 모델이 있는 경로>'
```

In [1]:
import os, sys
# clone한 경로 설정
sys.path.append('/home/cjh/D/dev/models/research/slim')
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import vgg

from urllib.request import urlopen
from datasets import dataset_utils, imagenet
from preprocessing import vgg_preprocessing

In [2]:
########################
# 0. 사전학습된 vgg16 경로 설정
# Pre-trained vgg16
target_dir = '../data/vgg16'

########################
# 1. 샘플 이미지를 vgg16 입력에 맞게 설정
# sample image
url = 'http://54.68.5.226/car.jpg'

img_as_string = urlopen(url).read()
image = tf.image.decode_jpeg(img_as_string, channels=3)

image_size = vgg.vgg_16.default_image_size  # image_size = 224

# vgg_preprocessing을 이용해 전처리 수행
processed_img = vgg_preprocessing.preprocess_image(image,
                                                   image_size,
                                                   image_size,
                                                   is_training=False)

processed_images = tf.expand_dims(processed_img, 0)

#########################
# 2. slim.arg_scope을 이용해 vgg16 예측값 설정
def vgg_arg_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        activation_fn=tf.nn.relu,
                        weights_regularizer=slim.l2_regularizer(weight_decay), 
                        biases_initializer=tf.zeros_initializer):
        with slim.arg_scope([slim.conv2d], padding='SAME') as arg_sc:
            return arg_sc

with slim.arg_scope(vgg.vgg_arg_scope()):
    logits, _ = vgg.vgg_16(processed_images, 
                           num_classes=1000, 
                           is_training=False)
probabilities = tf.nn.softmax(logits)

##########################
# 3. 사전 학습된 vgg16 모델 불러오기
load_vars = slim.assign_from_checkpoint_fn(
    os.path.join(target_dir, 'vgg_16.ckpt'),
    slim.get_model_variables('vgg_16'))


##########################
# 4. 추론하기
with tf.Session() as sess:
    load_vars(sess)
    network_input, probabilities = sess.run([processed_images,
                                            probabilities])
    probabilities = probabilities[0, 0:]
    
    names_ = imagenet.create_readable_names_for_imagenet_labels()
    
    idxs = np.argsort(-probabilities)[:5]
    probs = probabilities[idxs]
    classes = np.array(list(names_.values()))[idxs+1]
    for c, p in zip(classes, probs):
        print('Class: '+ c + '\t|Prob: ' + str(p))

INFO:tensorflow:Restoring parameters from ../data/vgg16/vgg_16.ckpt
Class: sports car, sport car	|Prob: 0.6969962
Class: car wheel	|Prob: 0.09234831
Class: convertible	|Prob: 0.0891054
Class: racer, race car, racing car	|Prob: 0.08648531
Class: beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon	|Prob: 0.011633869


### 7.5.5 정리

이번 포스팅에서는 TF-Slim에 대해 살펴보았다. TF-Slim은 CNN 모델링에 있어서 간편하게 작성할 수 있도록 다양한 계층등을 추상화해서 제공하며, AlexNet, VGG, Inception 등 성능이 좋은 사전 학습된 모델들을 제공한다. TF-Slim에 대해 자세한 설명은 https://github.com/tensorflow/tensorflow/tree/r1.8/tensorflow/contrib/slim 에서 확인할 수 있다.